### Week 1 - Basic Data Structures

#### 1. Check Brackets in the code

In [ ]:
from collections import namedtuple

Bracket = namedtuple("Bracket", ["char", "position"])

def are_matching(left, right):
    return (left + right) in ["()", "[]", "{}"]


def find_mismatch(text):
    opening_brackets_stack = []
    for i, char in enumerate(text):
        if char in "([{":
            # Process opening bracket
            Bracket = (char, i+1) 
            opening_brackets_stack.append(Bracket)

        if char in ")]}":
            # Process closing bracket
            if opening_brackets_stack == []:
                return i + 1
            top = opening_brackets_stack.pop()
            if not are_matching(top[0], char):                
                return i + 1
    if opening_brackets_stack == []:
        return "Success"
    else:
        return opening_brackets_stack[0][1]


In [ ]:
count = 1
for (text, answer) in [
    ("[]", "Success"),
    ("{}[]", "Success"),
    ("[()]", "Success"),
    ("(())", "Success"),
    ("{[]}()", "Success"),
    ("{", 1),
    ("}", 1),
    (" [}]", 3),
    ("foo(bar);", "Success"),
    ("foo(bar[i);",10 ),
    ("[](()", 3)
    ]:
    print("Test: ", count, text, find_mismatch(text) == answer)
    count += 1

#### 2. Compute Tree Height

In [ ]:
# python3

import sys
import threading


def compute_height_naive(n, parents):
    # Replace this code with a faster implementation
    max_height = 0
    for vertex in range(n):
        height = 0
        current = vertex
        while current != -1:
            height += 1
            current = parents[current]
        max_height = max(max_height, height)
    return max_height

In [ ]:
# python3

import sys
import threading

class TreeNode:
    """
        Single Tree Node 
    """
    # constructor function
    def __init__(self, key, parent=None):
        self.key = key
        self.parent = parent
        self.children = []
    
    # update parent node
    def update_parent(self, parent):
        self.parent = parent
        
    # add child to list of children nodes
    def add_child(self, child):
        self.children.append(child)
        
def build_tree(parents):
    n = len(parents)
    nodes = [None] * n
    for i in range(n):
        nodes[i] = TreeNode(i, parents[i])
    
    for child_index in range(n):
        parent_index = nodes[child_index].parent
        if parent_index == -1:
            root = child_index
        else:
            nodes[parent_index].add_child(child_index)
    
    return nodes, root
        

def compute_height(root, nodes):
    
    # base case, 0 nodes means height of 1
    if len(nodes[root].children) == 0:
        return 1
    
    max_height = 0
    for i in range(len(nodes[root].children)):
        max_height =  max(max_height, compute_height(nodes[root].children[i], nodes) + 1)

    return max_height

In [ ]:
# tree1
nodes = [4, -1, 4, 1, 1]
n = 5

nodes, root = build_tree(nodes)

height = compute_height(root, nodes)

print(height)


In [ ]:
# tree2
nodes = [-1, 0, 4, 0, 3]

nodes, root = build_tree(nodes)

height = compute_height(root, nodes)

print(height)

In [ ]:
# python3
# Couldn't get this verison optimized for large trees
import sys
import threading

parent_cache = dict()

def build_tree(node, nodes):
    parent = {'key': node, 'children': []}
    children = [n for n, x in enumerate(nodes) if parent['key'] == x]
    
    if parent['key'] not in parent_cache:
        for child in children:
            parent['children'].append(build_tree(child, nodes))
        parent_cache[parent['key']] = parent['children']
    else:
        parent['children'] = parent_cache['children']
        
    return parent
            
    
    #for child in children:
    #    parent['children'].append(build_tree(child, nodes))
    #return parent

tree_cache = dict()

def compute_height(tree):
    children = tree['children']
    if tree['key'] not in tree_cache:
        heights = (compute_height(child) for child in children)
        height = 1 +  max(heights, default=-1)
        #add height to the cache
        tree_cache[tree['key']] = height
    else:
        height = tree_cache['height'] 

    return height

#    return 1 + max ((compute_height(child) for child in children), default=-1)

# Helpful: 
# https://codereview.stackexchange.com/questions/128635/finding-the-tree-height-in-python
    
## Need to explore
### - adding memoization 
### - a DP approach (iterative + queue)

In [ ]:
parent_cache = dict()
tree_cache = dict()
parents = [4, -1, 4, 1, 1]

tree = build_tree(-1, parents)
print( compute_height(tree))

In [ ]:
parent_cache = dict()
tree_cache = dict()

parents_2 = [-1, 0, 4, 0, 3]

print(compute_height(build_tree(-1, parents_2)))

In [ ]:
def build_tree(root, nodes):
    children = [
        build_tree(child, nodes) for child, node in enumerate(nodes)]

#### 3. Network Simulation

In [168]:
# python3

from collections import namedtuple
from collections import deque

Request = namedtuple("Request", ["arrived_at", "time_to_process"])
Response = namedtuple("Response", ["was_dropped", "started_at"])


class Buffer:
    def __init__(self, size):
        self.size = size
        self.finish_time = deque()
        
    def process(self, request):
        
        # Remove processed items from buffer
        while len(self.finish_time) > 0:

            if self.finish_time[0] <= request.arrived_at:
                self.finish_time.popleft()
            else:
                break
        
        # Discard packet if buffer is full
        if len(self.finish_time) >= self.size:
            return Response (True, -1)
     
        # Empty buffer
        if len(self.finish_time) == 0:
            self.finish_time.append(request.arrived_at + request.time_to_process)
            return Response(False, request.arrived_at)
        
        last_finish_time = self.finish_time[-1]
        self.finish_time.append(last_finish_time + request.time_to_process)
        
        return Response(False, last_finish_time) 

def process_requests(requests, buffer):
    responses = []
    for request in requests:
        responses.append(buffer.process(request))
    return responses

def main():
    buffer_size, n_requests = map(int, input().split())
    requests = []
    for _ in range(n_requests):
        arrived_at, time_to_process = map(int, input().split())
        requests.append(Request(arrived_at, time_to_process))

    buffer = Buffer(buffer_size)
    responses = process_requests(requests, buffer)

    for response in responses:
        print(response.started_at if not response.was_dropped else -1)


#if __name__ == "__main__":
#    main()


In [160]:
# Sample 1
# Expect no output
# 

#requests = [Request(1,0)]
requests = []

buffer_size = 1
#print(requests)
#print(buffer_size)

buffer = Buffer(buffer_size)

responses = process_requests(requests, buffer)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

In [161]:
# Sample 2
# Expect:
# 0

arrived_at = 0
time_to_process = 1

requests = [Request(arrived_at,time_to_process)]

buffer_size = 1
#print(requests)
#print(buffer_size)

buffer = Buffer(buffer_size)
responses = process_requests(requests, buffer)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

0


In [162]:
# Sample 3
# Expect:
# 0
# -1
arrived_at = 0
time_to_process = 1

requests = [Request(arrived_at,time_to_process)]

arrived_at = 0
time_to_process = 1

requests.append(Request(arrived_at, time_to_process))

buffer_size = 1
#print(requests)
#print(buffer_size)

buffer = Buffer(buffer_size)
responses = process_requests(requests, buffer)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

0
-1


In [163]:
# Sample 4 
# Expect:
# 0
# 1
arrived_at = 0
time_to_process = 1

requests = [Request(arrived_at,time_to_process)]

arrived_at = 1
time_to_process = 1

requests.append(Request(arrived_at, time_to_process))

buffer_size = 1
#print(requests)
#print(buffer_size)

buffer = Buffer(buffer_size)
responses = process_requests(requests, buffer)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

0
1


In [164]:
# Sample 5
# Expect:
# 0
# 0

arrived_at = 0
time_to_process = 0

requests = [Request(arrived_at,time_to_process)]

arrived_at = 0
time_to_process = 0

requests.append(Request(arrived_at, time_to_process))

buffer_size = 1
#print(requests)
#print(buffer_size)

buffer = Buffer(buffer_size)
responses = process_requests(requests, buffer)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

0
0


In [165]:
# Sample 6
# Expect:
# 0
# 0

arrived_at = 0
time_to_process = 0

requests = [Request(arrived_at,time_to_process)]

arrived_at = 0
time_to_process = 1

requests.append(Request(arrived_at, time_to_process))

buffer_size = 1
#print(requests)
#print(buffer_size)

buffer = Buffer(buffer_size)
responses = process_requests(requests, buffer)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

0
0


In [166]:
# Sample 7
# Expect:
# 0
# 2

arrived_at = 0
time_to_process = 1

requests = [Request(arrived_at,time_to_process)]

arrived_at = 2
time_to_process = 1

requests.append(Request(arrived_at, time_to_process))

buffer_size = 1
#print(requests)
#print(buffer_size)

buffer = Buffer(buffer_size)
responses = process_requests(requests, buffer)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

0
2


In [167]:
# Sample 8
# Expect:
# 0
# 1

arrived_at = 0
time_to_process = 1

requests = [Request(arrived_at,time_to_process)]

arrived_at = 0
time_to_process = 1

requests.append(Request(arrived_at, time_to_process))

buffer_size = 2
#print(requests)
#print(buffer_size)

buffer = Buffer(buffer_size)
responses = process_requests(requests, buffer)

for response in responses:
    print(response.started_at if not response.was_dropped else -1)

0
1


#### 4. Extending Stack Interface